# Generating Word Embeddings - Lab

## Introduction

In this lab, we'll learn how to generate our own word embeddings by training our own Word2Vec model, and also by building embedding layers right into our Deep Neural Networks!

## Objectives

You will be able to:

* Demonstrate a basic understanding of the architecture of the Word2Vec model
* Demonstrate an understanding of the various tunable parameters of word2vec such as vector size and window size

## Getting Started

In this lab, we'll start by creating our own word embeddings by making use of the Word2Vec Model. Then, we'll move onto building Neural Networks that make use of **_Embedding Layers_** to accomplish the same end-goal, but directly in our model. 

The easiest way to make use of Word2Vec is to import it from the [Gensim Library](https://radimrehurek.com/gensim/). This model contains a full implementation of Word2Vec, which we can use to begin training immediately. For this lab, we'll be working with the [News Category Dataset from Kaggle](https://www.kaggle.com/rmisra/news-category-dataset/version/2#_=_).  This dataset contains headlines and article descriptions from the news, as well as categories for which type of article they belong to.  In this lab, we'll learn how to train a Word2Vec model on the text data to generate word embeddings for them. In the next lab, we'll then use the vectors created by our Word2Vec model to effectively train a classifier to predict the category of news given the headline and description of each article. In this lab, we won't do any classification, although we will learn how to train a Word2Vec model and explore the relationships between different word vectors in our embedding!

Run the cell below to import everything we'll need for this lab. 

In [2]:
!pip install gensim

    100% |████████████████████████████████| 24.6MB 2.2MB/s eta 0:00:01   31% |██████████▏                     | 7.8MB 15.7MB/s eta 0:00:02
    100% |████████████████████████████████| 40kB 20.3MB/s ta 0:00:01
    100% |████████████████████████████████| 133kB 19.1MB/s ta 0:00:01
    100% |████████████████████████████████| 5.3MB 4.1MB/s eta 0:00:01
    100% |████████████████████████████████| 71kB 22.1MB/s ta 0:00:01
  Running setup.py bdist_wheel for smart-open ... done
  Stored in directory: /Users/steeznation/Library/Caches/pip/wheels/f7/a6/ff/9ab5842c14e50e95a06a4675b0b4a689c9cab6064dac2b01d0
  Running setup.py bdist_wheel for bz2file ... done
  Stored in directory: /Users/steeznation/Library/Caches/pip/wheels/81/75/d6/e1317bf09bf1af5a30befc2a007869fa6e1f516b8f7c591cb9
Successfully built smart-open bz2file


In [6]:
import pandas as pd
import numpy as np
np.random.seed(0)
from gensim.models import Word2Vec
from nltk import word_tokenize

Now, we'll import the data. You'll find the data stored in the file `'News_Category_Dataset_v2.json'`.  This file is compressed, so that it can be more easily stored in a github repo. **_Make sure to unzip the file before continuing!_**

In the cell below, use the `read_json` function from pandas to read the dataset into a DataFrame. Be sure to also include the parameter `lines=True` when reading in the dataset!

Once you've loaded in the data, inspect the head of the DataFrame to see what our data looks like. 

In [5]:
df = pd.read_json('News_Category_Dataset_v2.zip', lines=True)
df.head()

,authors,category,date,headline,link,short_description
0,Melissa Jeltsen,CRIME,2018-05-26,There Were 2 Mass Shootings In Texas Last Week...,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...
1,Andy McDonald,ENTERTAINMENT,2018-05-26,Will Smith Joins Diplo And Nicky Jam For The 2...,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.
2,Ron Dicker,ENTERTAINMENT,2018-05-26,Hugh Grant Marries For The First Time At Age 57,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...
3,Ron Dicker,ENTERTAINMENT,2018-05-26,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...
4,Ron Dicker,ENTERTAINMENT,2018-05-26,Julianna Margulies Uses Donald Trump Poop Bags...,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ..."


## Preparing the Data

Since we're working with text data, we'll still need to do some basic preprocessing and tokenize our data. You'll notice from the sample of the data above that two different columns contain text data--`headline` and `short_description`. The more text data our Word2Vec model has, the better it will perform. Therefore, we'll want to combine the two columns before tokenizing each comment and training our Word2Vec model. 

In the cell below:

* Create a column called `combined_text` that consists of the data from `df.headline` plus a space character (`' '`) plus the data from `df.short_description`.
* Use the `combined_text` column's `map()` function and pass in `word_tokenize`. Store the result returned in `data`.

In [9]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/steeznation/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [10]:
df['combined_text'] = df.headline + ' ' + df.short_description
data = df['combined_text'].map(word_tokenize)

Let's inspect the first 5 items in `data` to see how everything looks. 

In [11]:
data[:5]

0    [There, Were, 2, Mass, Shootings, In, Texas, L...
1    [Will, Smith, Joins, Diplo, And, Nicky, Jam, F...
2    [Hugh, Grant, Marries, For, The, First, Time, ...
3    [Jim, Carrey, Blasts, 'Castrato, ', Adam, Schi...
4    [Julianna, Margulies, Uses, Donald, Trump, Poo...
Name: combined_text, dtype: object

You'll notice that although the words are tokenized, they are still in the same order they were in as headlines. This is important, because the words need to be in their original order for Word2Vec to establish the meaning of them. Recall from our previous lesson on how Word2Vec works that we can specify a  **_Window Size_** that tells the model how many words to take into consideration at one time. 

If our window size was 5, then the model would start by looking at the words "Will Smith joins Diplo and", and then slide the window by one, so that it's looking at "Smith joins Diplo and Nicky", and so on, until it had completely processed the text example at index 1 above. By doing this for every piece of text in the entire dataset, the Word2Vec model learns excellent vector representations for each word in an **_Embedding Space_**, where the relationships between vectors capture semantic meaning (recall the vector that captures gender in the previous "king - man + woman = queen" example we saw).

Now that we've prepared our data, let's train our model and explore a bit!

## Training the Model

We'll start by instantiating a Word2Vec Model from gensim below. 

In the cell below:

* Create a `Word2Vec` model and pass in the following arguments:
    * The dataset we'll be training on, `data`
    * The size of the word vectors to create, `size=100`
    * The window size, `window=5`
    * The minimum number of times a word needs to appear in order to be counted in  the model, `min_count=1`.
    * The number of threads to use during training, `workers=4`

In [12]:
model = Word2Vec(data, size=100, window=5, min_count=1, workers=4)

Now, that we've created our Word2Vec model, we still need to train it on our model. 

In the cell below:

* Call `model.train()` and pass in the following parameters:
    * The dataset we'll be training on, `data`
    * The `total_examples`  of sentences in the dataset, which we can find in `model.corpus_count`. 
    * The number of `epochs` we want to train for, which we'll set to `10`

In [13]:
model.train(data, total_examples=model.corpus_count, epochs=10)

(55563525, 67352790)

Great! We now have a fully trained model! The word vectors themselves are stored inside of a `Word2VecKeyedVectors` instance, which we'll find stored inside of `model.wv`. For simplicity's sake, let's go ahead and store this inside of the variable `wv` in order to save ourselves some keystrokes down the line. 

In [14]:
wv = model.wv

## Examining Our Word Vectors

Now that we have a trained Word2Vec model, let's go ahead and explore the relationships between some of the words in our corpus! 

One cool thing we can use Word2Vec for is to get the most similar words to a given word. We can do this passing in the word to `wv.most_similar()`. 

In the cell below, let's try getting the most similar word to `'Texas'`.

In [15]:
wv.most_similar('Texas')

[('Oregon', 0.8150627017021179),
 ('Oklahoma', 0.8035823106765747),
 ('Pennsylvania', 0.8004339933395386),
 ('Florida', 0.7992469668388367),
 ('Louisiana', 0.7961402535438538),
 ('Illinois', 0.7927291393280029),
 ('Ohio', 0.7918649911880493),
 ('California', 0.7904971837997437),
 ('Maryland', 0.7882695198059082),
 ('Connecticut', 0.7869223356246948)]

Interesting! All of the most similar words are also states. 

We can also get the least similar vectors to a given word by passing in the word to the `most_similar()` function's `negative` parameter. 

In the cell below, get the least similar words to `'Texas'`.

In [16]:
wv.most_similar(negative='Texas')

[('once-reliable', 0.42014172673225403),
 ('Specious', 0.40487539768218994),
 ('Parent/Grandparent', 0.3984057307243347),
 ('Likened', 0.3600238859653473),
 ('Strategist™', 0.35930880904197693),
 ('confidantes', 0.357951283454895),
 ('Relational', 0.35010528564453125),
 ('maitre', 0.34368276596069336),
 ('Hunger-Free', 0.33978772163391113),
 ('Palatetraining', 0.3355695605278015)]

These seem like just noise. This is because of the way Word2Vec is computing the similarity between word vectors in the embedding space. Although the word vectors closest to a given word vector are almost certainly going to have similar meaning or connotation with our given word, the word vectors that the model considers 'least similar' are just the word vectors that are farthest away, or have the lowest cosine similarity. It's important to understand that while the closest vectors in the embedding space will almost certainly share some level of semantic meaning with a given word, there is no guarantee that this relationship will hold at large distances. 

We can also get the vector for a given word by passing in the word as if we were passing in a key to a dictionary. 

In the cell below, get the word vector for `'Texas'`.

In [17]:
wv['Texas']

array([-0.45584056,  0.4978539 , -2.8317404 , -1.6143125 ,  0.66161114,
       -2.824803  ,  0.8131217 ,  0.2899068 ,  0.02442012,  0.00643601,
        0.61297226,  0.25130728,  0.93771446, -1.4267892 , -0.5837486 ,
       -1.0189909 ,  0.27375737,  0.96674293, -0.90405834, -0.8898436 ,
        1.6105971 ,  0.7857851 ,  1.2630434 ,  0.72980237,  2.1884964 ,
        1.0333041 ,  1.6379173 ,  0.32369736, -1.1321595 , -2.7063713 ,
        1.7278571 ,  0.18838291,  1.7467893 , -0.88902134, -0.01822575,
       -1.169414  , -0.10303354,  2.2586124 ,  0.41613078,  0.60748965,
        1.1600393 , -1.1639131 , -1.7272866 , -2.9046056 , -0.04696259,
       -1.0886884 , -0.33500677,  1.2305641 ,  1.7118844 ,  1.3769999 ,
        0.31261805,  0.11066099,  1.6837397 , -0.9000292 ,  2.3175752 ,
       -2.286697  , -0.6769622 ,  0.04617644,  0.8357025 , -1.3828776 ,
        1.013813  ,  0.62213224, -1.0747435 ,  0.74921596,  1.321444  ,
       -0.7926867 , -1.1885496 ,  1.106186  ,  0.22342096,  0.87

Let's get all of the word vectors from the object at once. We can find these inside of `wv.vectors`.  Do this now in the cell below.  

In [18]:
wv.vectors

array([[ 1.1452972 ,  1.0862144 ,  1.0799356 , ..., -0.02128634,
         0.3938028 , -0.36351377],
       [ 2.5094662 ,  0.5767207 ,  1.279885  , ..., -1.5726373 ,
         0.6729144 , -0.75653404],
       [ 0.7565386 ,  0.01826349, -0.0835297 , ...,  0.2808548 ,
         0.8054805 , -1.9590547 ],
       ...,
       [-0.01449146, -0.00764316, -0.0108026 , ...,  0.01657663,
        -0.00738603,  0.05902426],
       [ 0.09696476,  0.1361734 ,  0.03161414, ...,  0.0356192 ,
        -0.06789645,  0.05407048],
       [ 0.02690641, -0.01121263, -0.04974168, ..., -0.00328935,
         0.00523712,  0.04192706]], dtype=float32)

As a final exercise, let's try recreating the _'king' - 'man' + 'woman' = 'queen'_ example we've seen before. We can do this by using the `most_similar` function and putting the things we want added together inside of an array passed to the `positive` parameter, and the things we want subtracted as an array passed to the the `negative` parameter. 

Do this now in the cell below. 

In [20]:
wv.most_similar(positive=['king', 'woman'], negative=['man'])

[('title', 0.6252315640449524),
 ('revival', 0.6014490127563477),
 ('queen', 0.5931322574615479),
 ('adaptation', 0.5846633315086365),
 ('reminiscent', 0.581926703453064),
 ('clutch', 0.5765603184700012),
 ('goddess', 0.5744433403015137),
 ('novel', 0.5738402605056763),
 ('villain', 0.5731254816055298),
 ('princess', 0.5728788375854492)]

As we can see from the output above, our model isn't perfect, but 'Queen' is still in the top 3, and with 'Princess' not too far behind. As we can see from the word in first place, 'reminiscent', our model is far from perfect. This is likely because we didn't give it too much training, or training data. However, for the small amount of training data it was given, the model still performs remarkably well! 

We'll see in the next lab that from a practical standpoint, one of the best things we can do for performance is to start by loading in the weights from an open-sourced model that has been trained for a very long time on a massive amount of data, such as the GloVe model from the Stanford NLP Group. There's not really any benefit from training the model ourselves, unless our text uses different, specialized vocabulary that isn't likely to be well represented inside an open-source model.

## Summary

In this lab, we learned how to train and use a Word2Vec model to created vectorized word embeddings!